# LoRA Fine-Tuning with SmolLM2-360M

**Base Model:** HuggingFaceTB/SmolLM2-360M-Instruct  
**Dataset:** shawhin/imdb-truncated (1000 train, 1000 validation samples)

## 1. Setup and Installation

In [21]:
# Install required packages
!pip install -q transformers datasets peft accelerate bitsandbytes trl torch

In [ ]:
# Import from our lora_finetuning module
from lora_finetuning import (
    setup_lora_model,
    prepare_dataset,
    train_lora_model,
    evaluate_model,
    generate_response,
    load_finetuned_model
)

import torch
import numpy as np
from datasets import load_dataset

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

## 2. Load Dataset and Inspect

In [23]:
# Load the IMDB dataset
dataset = load_dataset('shawhin/imdb-truncated')
print(dataset)
print("\nSample from training set:")
print(dataset['train'][0])

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 1000
    })
})

Sample from training set:
{'label': 1, 'text': '. . . or type on a computer keyboard, they\'d probably give this eponymous film a rating of "10." After all, no elephants are shown being killed during the movie; it is not even implied that any are hurt. To the contrary, the master of ELEPHANT WALK, John Wiley (Peter Finch), complains that he cannot shoot any of the pachyderms--no matter how menacing--without a permit from the government (and his tone suggests such permits are not within the realm of probability). Furthermore, the elements conspire--in the form of an unusual drought and a human cholera epidemic--to leave the Wiley plantation house vulnerable to total destruction by the Elephant People (as the natives dub them) to close the story. If you happen to see the current release EARTH

## 3. Load Base Model and Tokenizer

In [ ]:
# Model configuration
model_name = "HuggingFaceTB/SmolLM2-360M-Instruct"

# Import necessary classes for base model loading
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set padding token if not present
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

# Load base model (without LoRA for initial testing)
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

print(f"Model loaded: {model_name}")
print(f"Model parameters: {base_model.num_parameters():,}")

## 4. Test Base Model (Before Fine-Tuning)

Let's evaluate the base model on a subset of the validation dataset to establish a baseline.

In [ ]:
# Using imported functions from lora_finetuning module
# - generate_response()
# - evaluate_model()

print("=" * 80)
print("BASE MODEL EVALUATION (Before Fine-Tuning)")
print("=" * 80)
print()

base_acc, base_correct, base_total = evaluate_model(base_model, tokenizer, dataset, num_samples=100, debug=True)

print("\n" + "=" * 80)
print(f"BASE MODEL RESULTS:")
print(f"Accuracy: {base_acc:.2%} ({base_correct}/{base_total} correct)")
print("=" * 80)

## 5. Prepare Dataset for Fine-Tuning

Format the IMDB dataset for sentiment analysis training.

In [ ]:
# Using the imported prepare_dataset() function
# This function handles:
# - Creating prompts with create_prompt()
# - Tokenizing with tokenize_function()
# - Returning train, validation, and raw datasets

print("Preparing dataset...")
tokenized_train, tokenized_val, dataset = prepare_dataset(
    dataset_name='shawhin/imdb-truncated',
    tokenizer=tokenizer,
    max_length=256
)

print(f"\nTokenized training samples: {len(tokenized_train)}")
print(f"Tokenized validation samples: {len(tokenized_val)}")

print("\nSample from formatted dataset:")
print("(The raw text has been converted to instruction-formatted prompts)")

In [ ]:
# Dataset preparation is now complete!
# The prepare_dataset() function has already:
# 1. Loaded the dataset
# 2. Created instruction-formatted prompts
# 3. Tokenized the text with max_length=256
# 4. Added labels for training

print("✓ Dataset ready for training")
print(f"  Training samples: {len(tokenized_train)}")
print(f"  Validation samples: {len(tokenized_val)}")

## 6. Configure LoRA and PEFT

Set up Low-Rank Adaptation (LoRA) configuration for efficient fine-tuning.

In [ ]:
# Using setup_lora_model() to configure and apply LoRA
# This function handles:
# - Loading the base model
# - Creating LoRA configuration
# - Applying PEFT/LoRA to the model
# - Enabling gradient checkpointing

print("Setting up model with LoRA configuration...")

model, tokenizer, lora_config = setup_lora_model(
    model_name=model_name,
    lora_r=16,                 # Rank of the low-rank matrices
    lora_alpha=32,             # Scaling factor
    lora_dropout=0.05,         # Dropout probability
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"]  # Modules to apply LoRA
)

print("\n✓ Model ready for fine-tuning with LoRA")

In [ ]:
# The setup_lora_model() function has completed all setup steps:
# - Loaded base model with float16 precision
# - Configured LoRA (r=16, alpha=32, dropout=0.05)
# - Applied LoRA to attention projection layers
# - Enabled gradient checkpointing for memory efficiency
# - Prepared model for k-bit training

print("Model configuration complete!")
print(f"Ready to train {model.num_parameters():,} parameters")
print(f"(Only {model.get_nb_trainable_parameters()} are trainable with LoRA)")

## 7. Configure Training Arguments and Trainer

In [ ]:
# Using train_lora_model() to train the model
# This function handles:
# - Setting up training arguments
# - Creating data collator
# - Initializing Trainer
# - Running the training loop

output_dir = "./lora_finetuned_smollm2"

trainer = train_lora_model(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    output_dir=output_dir,
    num_epochs=3,
    batch_size=4,
    learning_rate=2e-4,
    gradient_accumulation_steps=4,
    warmup_steps=100,
    logging_steps=50
)

print("\n✓ Training configuration complete, ready to train!")

## 8. Train the Model

This will take several minutes depending on your hardware.

In [ ]:
# Training has been completed by train_lora_model()!
# The function already called trainer.train() and displayed progress

print("=" * 80)
print("Training Summary:")
print("=" * 80)
print(f"✓ Model trained for 3 epochs")
print(f"✓ Best model saved to: {output_dir}")
print(f"✓ Training logs available in trainer.state")
print("=" * 80)

## 9. Test Fine-Tuned Model and Compare

Load the fine-tuned model and compare its performance with the base model on the same validation samples.

In [ ]:
# Using load_finetuned_model() to load the trained model
# This function handles:
# - Loading the base model
# - Loading and applying the LoRA adapter weights

print("Loading fine-tuned model...")

# The model is already in memory from training, but let's demonstrate
# how to load it fresh (useful when restarting the notebook)
lora_adapter_path = output_dir

finetuned_model = load_finetuned_model(
    base_model_name=model_name,
    adapter_path=lora_adapter_path
)

print("Fine-tuned model loaded successfully!")

In [ ]:
# Using evaluate_model() to test the fine-tuned model
print("=" * 80)
print("FINE-TUNED MODEL EVALUATION")
print("=" * 80)
print()

ft_acc, ft_correct, ft_total = evaluate_model(finetuned_model, tokenizer, dataset, num_samples=100)

print("\n" + "=" * 80)
print(f"FINE-TUNED MODEL RESULTS:")
print(f"Accuracy: {ft_acc:.2%} ({ft_correct}/{ft_total} correct)")
print("=" * 80)

## 10. Compare Results

In [40]:
print("\n" + "=" * 80)
print("MODEL COMPARISON: Base vs Fine-Tuned")
print("=" * 80)
print(f"\nBase Model Accuracy:       {base_acc:.2%} ({base_correct}/{base_total})")
print(f"Fine-Tuned Model Accuracy: {ft_acc:.2%} ({ft_correct}/{ft_total})")
print(f"\nAbsolute Improvement:      {(ft_acc - base_acc):.2%}")
print(f"Relative Improvement:      {((ft_acc - base_acc) / base_acc * 100):.1f}%")
print("=" * 80)



MODEL COMPARISON: Base vs Fine-Tuned

Base Model Accuracy:       66.00% (66/100)
Fine-Tuned Model Accuracy: 82.00% (82/100)

Absolute Improvement:      16.00%
Relative Improvement:      24.2%
